In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from qiskit.ignis.verification.tomography import state_tomography_circuits
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Operator
from sympy import Matrix


/var/folders/gq/pyfb45qx3wg52k7kygqcjy840000gn/T/ipykernel_1555/2181746857.py:4: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import state_tomography_circuits


In [3]:
def random_Operator_matrix(N_qubits):
    #### generate a random unistocastic matrix
    t = 1.0
    A = np.random.uniform(0,1, size=(2**(N_qubits),2**(N_qubits)))
    B = np.random.uniform(0,1, size=(2**(N_qubits),2**(N_qubits)))
    H = np.zeros((2**N_qubits,2**N_qubits), dtype=complex)
    H = A + 1j*B
    H = H + np.conjugate(H).T
    U = expm(1j*H*t)
    return U

def random_Quantum_state(N_qubits):
    #### generate a random unistocastic matrix
    return random_Operator_matrix(N_qubits=N_qubits)[:,0]

def random_Probability_matrix(N_qubits):
    #### for a random probability matrix
    probability_matrix = np.zeros((2**N_qubits,2**N_qubits))
    for i in range(2**N_qubits):
        column = np.random.uniform(0,1, size=(2**N_qubits))
        probability_matrix[:,i] = column/sum(column)
    return probability_matrix

def prob_matrix_check(matrix, decimals):
    col = [np.round(sum(matrix[:,i]), decimals=decimals)==1 for i in range(len(matrix))]
    row = [np.round(sum(matrix[i,:]), decimals=decimals)==1 for i in range(len(matrix))]

    if all(col):
        if all(row):
            return True, True
        else:
            return False, True
    elif all(row):
        return True, False
    return False, False

In [4]:
N_qubits = 2

# generate a random probability matrix
prob_matrix = random_Probability_matrix(N_qubits=N_qubits)
oper_matrix = random_Operator_matrix(N_qubits=N_qubits)

U_tilde = prob_matrix
Matrix(U_tilde)

Matrix([
[ 0.554357792276899,  0.496726991991143, 0.0123724955451327, 0.265675109738258],
[ 0.138554110872133,  0.429478864957671,  0.697614219269808, 0.261823188103243],
[ 0.280660700226604,  0.062701742060106,  0.255920898500188, 0.061036734213767],
[0.0264273966243639, 0.0110924009910796, 0.0340923866848713, 0.411464967944733]])

In [13]:
C_CIC_ideal = oper_matrix
C_CIC = U_tilde*np.abs(np.linalg.inv(U_tilde))**2
Matrix(C_CIC)

Matrix([
[  0.183546020181778,   0.598277525166759,   0.109806794445204, 0.00352032524205763],
[  0.268452457977562,   0.641802984032171,     7.3243549281171,   0.374494743770023],
[  0.266006434205146,  0.0522834123243918,   0.566988047404827, 0.00183212528938558],
[1.01003433739495e-6, 1.61249752787322e-5, 0.00176164887711104,     2.5395191628092]])

In [14]:
C_CIC = U_tilde*np.linalg.inv(np.abs(U_tilde)**2)
Matrix(C_CIC)

Matrix([
[  0.459328659136218,  -0.654133949714948,    0.121048969134178, -0.00730990424219926],
[   0.41856000995643,   0.704762529894607,     -8.4906216339779,   -0.433591119091587],
[ -0.330617016203237,  0.0931520417592708,     1.08396178894926,   -0.012428866736844],
[6.53711346897444e-5, -6.6102658696194e-5, -0.00206572161548995,     2.43145767630375]])

In [15]:
C_NIC = U_tilde

verifico che il prodotto di una matrice di ampiezze per uno stato quantistico modulo quadro è uguale al prodotto dei moduli quadri

In [16]:
psi = random_Quantum_state(N_qubits=N_qubits)
C = random_Operator_matrix(N_qubits=N_qubits)

Matrix(np.abs(np.dot(C, psi)))

Matrix([
[0.100184817486524],
[0.758740655942246],
[0.387596800340362],
[0.513852449377674]])

In [17]:
Matrix(np.dot(np.abs(C), np.abs(psi)))

Matrix([
[0.938316327147962],
[0.918562478921619],
[0.931641515245632],
[0.879901416554432]])

In [18]:
C_1 = random_Operator_matrix(N_qubits=N_qubits)
U = random_Operator_matrix(N_qubits=N_qubits)

In [19]:
Matrix(np.abs(np.dot(C_1,U))**2)

Matrix([
[0.0623365317745898, 0.0362116555597657,  0.479919364180364, 0.42153244848528],
[ 0.233814989803081,   0.28375176541933,  0.182204504872749, 0.30022873990484],
[ 0.530172718723082,  0.287575056703805, 0.0773646196347637, 0.10488760493835],
[ 0.173675759699248,    0.3924615223171,  0.260511511312123, 0.17335120667153]])

In [20]:
Matrix(np.dot(np.abs(C_1)**2,np.abs(U)**2))

Matrix([
[0.155866822131948, 0.102593781385337, 0.469807545537043, 0.271731850945671],
[0.346747248034752, 0.265449732860526,  0.16360246189093, 0.224200557213791],
[0.367067745795613, 0.231695001854189, 0.142000907149194, 0.259236345201003],
[0.130318184037686, 0.400261483899947, 0.224589085422832, 0.244831246639535]])